In [ ]:
import os
import os.path as op
import numpy as np
import pandas as pd

import nibabel as nib
from nimare.transforms import p_to_z

import nilearn
from nilearn import plotting
from nilearn import datasets
import nilearn.reporting
from nilearn.image import threshold_img

from nilearn.maskers import NiftiSpheresMasker

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data_dir = "./dset"
age_dir = op.join(data_dir, "age-effect5-21old/habenula")
clust_dir = op.join(data_dir, "age-effect/habenula")

In [ ]:
# get peaks
brik_fn = op.join(age_dir, "sub-group_task-rest_desc-agehabenula_briks+tlrc.BRIK")
table_fn = op.join(age_dir, "sub-group_task-rest_desc-Agehabenula_table.txt")
nii_1s_fn = op.join(age_dir, "sub-group_task-rest_desc-ageDiff_zmap.nii.gz")
cluster_fn = op.join(clust_dir, "clustsim", "clustsim_out.NN3_bisided.1D")

column_names = [".10000", ".05000", ".02000", ".01000"]
cluster_df = pd.read_table(
    cluster_fn, skiprows=8, delim_whitespace=True, names=column_names
)
cluster_df = cluster_df.reset_index()
cluster_df.rename(columns={"index": "pthr"}, inplace=True)
print(cluster_df)

cmap = "cool"
brik_idx = [11]
nii_fns = [nii_1s_fn]
tests = ["1s"]
alpha = ".10000"
pthrs = [0.0050]
cohen_thresh = 0

data_df = pd.read_csv(table_fn, sep="\t")
n_sub = data_df.groupby("group").size().sum()
n_sub_1, n_sub_2 = data_df.groupby("group").size().values

for brik_i, nii_fn, test, pthr in zip(brik_idx, nii_fns, tests, pthrs):
    convert = f"3dAFNItoNIFTI \
        -prefix {nii_fn} \
        {brik_fn}[{brik_i}]"
    os.system(convert)

    nii_img = nib.load(nii_fn)
    z_thresh = p_to_z(pthr)
    info = nii_img.get_fdata()

    clust_ext = cluster_df.loc[cluster_df["pthr"] == pthr, alpha].values[0]
    nii_thr_img = threshold_img(nii_img, z_thresh, cluster_threshold=clust_ext)
    print(clust_ext, pthr, z_thresh)

    clusters = nilearn.reporting.get_clusters_table(
        nii_thr_img, z_thresh)
    print(clusters) #coordinates are same as affine of input (MNI)

In [ ]:
# Filter the DataFrame to include only rows where 'Cluster Size (mm3)' is empty
subclusters = clusters[clusters["Cluster Size (mm3)"] == ""]

# Extract the main cluster ID from the 'Cluster ID' column for grouping
subclusters["Main Cluster ID"] = subclusters["Cluster ID"].str.extract(
    r"(\d+)", expand=False
)

# Group by the main cluster ID and extract coordinates
grouped_coordinates = subclusters.groupby("Main Cluster ID").apply(
    lambda df: df[["X", "Y", "Z"]].values.tolist()
)

# Convert the grouped coordinates to a dictionary for easier access
coordinates_dict = grouped_coordinates.to_dict()

# Print the coordinates dictionary
for cluster_id, coords in coordinates_dict.items():
    print(f"Cluster {cluster_id}: {coords}")

In [ ]:
# Get a list of all directories in the base directory
dset = os.listdir(data_dir)

# Filter out only subject directories (assuming they start with 'sub-')
sub_dir = [d for d in dset if d.startswith("sub-")]

# Loop over each subject directory
# on hpc will use partipant.tsv
for subject in sub_dir:
    subject_path = os.path.join(data_dir, subject, "func")

    if os.path.isdir(subject_path):
        print(f"Processing subject: {subject}")
        subject_brik = op.join(
            subject_path,
            f"{subject}_task-rest_space-MNI152NLin2009cAsym_res-2_desc-norm_bucketREML+tlrc.BRIK",
        )
        subject_nii = op.join(
            subject_path,
            f"{subject}_task-rest_space-MNI152NLin2009cAsym_res-2_desc-norm_bucketREML+tlrc.nii.gz",
        )

        convert = f"3dAFNItoNIFTI \
        -prefix {subject_nii} \
        {subject_brik}[{1}]"
        os.system(convert)

In [ ]:
results = []

for cluster_id, coords in coordinates_dict.items():
    print(f"Cluster {cluster_id}: {coords}")
    for subject in sub_dir:
        subject_path = os.path.join(data_dir, subject, "func")

        if os.path.isdir(subject_path):
            print(f"Processing subject: {subject}")

            subject_nii = nib.load(op.join(
                subject_path,
                f"{subject}_task-rest_space-MNI152NLin2009cAsym_res-2_desc-norm_bucketREML+tlrc.nii.gz",
            ))

            mask_img = datasets.load_mni152_brain_mask(resolution=1)

            for coord in coords:
                print(coord)
                masker = NiftiSpheresMasker([coord], radius = 5, mask_img=mask_img)
                masker = masker.fit_transform(subject_nii)
                print(masker)

                results.append(
                    {"Coordinate": coord, "Subject": subject, "Correlation": masker}
                )

corr_df = pd.DataFrame(results)
print(corr_df)

In [ ]:
for cluster_id, coords in coordinates_dict.items():
    print(f"Cluster {cluster_id}: {coords}")

coords_all = []
for coord_list in coordinates_dict.values():
    coords_all.extend(coord_list)

# Create an empty image
brain_mask = datasets.load_mni152_template(resolution=1)
spheres_img = np.zeros(brain_mask.shape)

# Fill the spheres in the empty image
affine = brain_mask.affine
for coord in coords_all:
    # Convert from MNI coordinates to voxel indices
    x, y, z = nib.affines.apply_affine(np.linalg.inv(affine), coord).astype(int)
    radius = 5  # radius in voxels

    # Create a sphere
    for i in range(-radius, radius + 1):
        for j in range(-radius, radius + 1):
            for k in range(-radius, radius + 1):
                if i**2 + j**2 + k**2 <= radius**2:
                    if (
                        0 <= x + i < spheres_img.shape[0]
                        and 0 <= y + j < spheres_img.shape[1]
                        and 0 <= z + k < spheres_img.shape[2]
                    ):
                        spheres_img[x + i, y + j, z + k] = 1

# Convert the numpy array to a NIfTI image
spheres_nifti = nib.Nifti1Image(spheres_img, affine)

# Plot the spheres on a glass brain
plotting.plot_glass_brain(
    spheres_nifti, display_mode="ortho", colorbar=True, title=f"{radius}mm Spheres for Age Effect", cmap='cool'
)

# Show the plot
plotting.show()

In [ ]:
corr_df = pd.read_csv(op.join(age_dir, "age-correlation.csv"))
corr_df["Coordinate"] = corr_df["Coordinate"].apply(eval)

for cluster_id, coords in coordinates_dict.items():
    for coord in coords:
        coord_df = corr_df[corr_df["Coordinate"].apply(lambda x: x == coord)]
        print(f"Condensed DataFrame for Coordinate {coord}:")
        print(coord_df)

        g = sns.lmplot(
            data=coord_df,
            x="Age",
            y="Correlation",
            hue="Group",
            height=7,
        )

        plt.ylim(-0.6, 0.6)
        
        # Add title to the plot
        plt.title(f"Age Effect: {radius}mm Sphere for MNI {coord}")

        # Show the plot
        plt.show()

In [ ]:
for cluster_id, coords in coordinates_dict.items():
    print(f"Cluster {cluster_id}: {coords}")

coords_all = []
for coord_list in coordinates_dict.values():
    coords_all.extend(coord_list)

# Create an empty image
brain_mask = datasets.load_mni152_template(resolution=1)
spheres_img = np.zeros(brain_mask.shape)

# Fill the spheres in the empty image
affine = brain_mask.affine
for coord in coords_all:
    # Convert from MNI coordinates to voxel indices
    x, y, z = nib.affines.apply_affine(np.linalg.inv(affine), coord).astype(int)
    radius = 5  # radius in voxels

    # Create a sphere
    for i in range(-radius, radius + 1):
        for j in range(-radius, radius + 1):
            for k in range(-radius, radius + 1):
                if i**2 + j**2 + k**2 <= radius**2:
                    if (
                        0 <= x + i < spheres_img.shape[0]
                        and 0 <= y + j < spheres_img.shape[1]
                        and 0 <= z + k < spheres_img.shape[2]
                    ):
                        spheres_img[x + i, y + j, z + k] = 1

# Convert the numpy array to a NIfTI image
spheres_nifti = nib.Nifti1Image(spheres_img, affine)

# Plot the spheres on a glass brain
plotting.plot_glass_brain(
    spheres_nifti, display_mode="ortho", colorbar=True, title=f"{radius}mm Spheres for Age Effect", cmap='cool'
)

# Show the plot
plotting.show()

In [ ]:
# Assuming corr_df is your DataFrame loaded from "age-correlation.csv"
corr_df = pd.read_csv(op.join(age_dir, "age-correlation.csv"))

# Group by 'Subject' and 'Cluster', and calculate the mean of 'Correlation'
corr_df_mean = corr_df.groupby(["Subject", "Cluster"], as_index=False)[
    "Correlation"
].mean()
corr_df_mean.rename(columns={"Correlation": "Avg Correlation"}, inplace=True)

# Merge averaged correlations back into the original DataFrame and replace 'Correlation' column
corr_df = pd.merge(corr_df, corr_df_mean, on=["Subject", "Cluster"], how="left")
corr_df["Correlation"] = corr_df[
    "Avg Correlation"
]  # Replace original 'Correlation' with 'Avg_Correlation'
corr_df.drop(columns=["Correlation", "Coordinate"], inplace=True)  # Drop the temporary column

corr_df.drop_duplicates(subset=["Subject", "Cluster"], keep="first", inplace=True)

print(corr_df)

clusters = [1, 2]
for cluster in clusters:
    cluster_df = corr_df[
        corr_df["Cluster"] == cluster
    ].copy()  # Filter for current cluster
    
    g = sns.lmplot(
            data=cluster_df,
            x="Age",
            y="Avg Correlation",
            hue="Group",
            height=7,
        )

    plt.ylim(-0.3, 0.3)
        
    # Add title to the plot
    plt.title(f"Age Effect: {radius}mm Sphere for Cluster {cluster}")

    # Show the plot
    plt.show()